In [3]:
import numpy as np

### Extrinsic Matrix calculation

In [4]:
def quaternion_rotation_matrix(Q, T):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
    
    tx = T[0]
    ty = T[1]
    tz = T[2]
    
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
    
    translation_matrix = np.array([[tx],
                                   [ty],
                                   [tz]])
                            
    transformation_matrix = getTransformation(rot_matrix, translation_matrix)
    return rot_matrix, translation_matrix, transformation_matrix

def getTransformation(R, t):
    # given Rotation matrix, R and translation matrix, t
    # return the transformation matrix describing the combined motion, T
    augment = np.array([0, 0, 0, 1]) # Converting to homogeneous matrix
    R = np.array(R)
    t = np.array(t)
    if (t.shape == (3,)):
        t = t.reshape((3,1))
    T = np.hstack((R, t))
    T = np.vstack((T, augment))
    assert(T.shape, (4,4))
    return T

# Inverse of a Transformation matrix
def getInverseTransform(T):
    # Given a transformation, T that moves(rotates and translates) a point from A
    # to B, return inv_T transformation that restores the point back to A.
    assert(T.shape, (4,4))
    R = T[:3,:3]
    print('R shape:', R.shape)
    R_inv = R.T
    print('R_inv shape:', R_inv.shape)
    t = T[:3,3]
    t_inv = -1 * np.matmul(R_inv, t)
    t_inv = t_inv.reshape(t_inv.shape[0], -1)
    print('t_inv shape:', t_inv.shape)
    T_inv = np.hstack((R_inv, t_inv))
    augment = np.array([0, 0, 0, 1]) # Converting to homogeneous matrix
    T_inv = np.vstack((T_inv, augment))
    return T_inv

def mapto3D(T, K, K_inv, coordMatrix2D):
    '''
        T: transformation matrix
        K: intrinsic matrix
        K_inv:  inverse of intrinsic matrix
        coordMatrix2D: 2d coordinates = [u v 1]^transpose
    
        coordMatrix3D: 3d coorinates = [x_w y_w z_w 1]^transpose
    '''
    print('T shape:', T.shape)
    coordMatrix3D = None
    T_inv = getInverseTransform(T)
    
    '''
    K_inv = K_inv
    try:
        K_inv = np.linalg.inv(K)
    '''
    
#     print('inv(K) shape:', np.linalg.inv(K).shape)
    print('K_inv shape:', K_inv.shape)
    coordMatrix3D = np.matmul(np.matmul(T_inv, K_inv), coordMatrix2D)
    
    '''
    except np.linalg.LinAlgError:
#         Not invertible. Skip this one.
    print('K is Singular matrix - non invertible')
        pass
    '''
    
    return coordMatrix3D

<>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-4-0a23afe0cebe>:60: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(T.shape, (4,4))
<ipython-input-4-0a23afe0cebe>:67: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(T.shape, (4,4))


### Intrinsic Matrix calculation
#### NOTE: Please cross-check below params once

In [5]:
'''
alphaX: focalLength in x in pixels
alphaY: focalLenght in y in pixels
u0 = principal point in X
v0 = principal point in y
gamma: skew coeff between x and y axis. is commonly 0
'''
alphaX = 960
alphaY = 960
u0 = 480
v0 = 360
gamma = 0

import sympy as sp

def determinant(A):
    if len(sp.Matrix(A).rref()[1]) < max(np.shape(A)):
        print('1st condition')
        return 0
    else:
        return np.linalg.det(A)

K = np.array([[alphaX, gamma, u0, 0],
              [0,     alphaY, v0, 0],
              [0,     0,      1,  0]])

K_inv = (1/(alphaX*alphaY))*np.array([[alphaY, gamma, -u0*alphaY, 0],
                                      [0,     alphaX, -v0*alphaX, 0],
                                      [0,     0,      alphaX*alphaY,  0]])
# print(determinant(K))
# print(np.linalg.det(K))
print(np.matmul(K.T,K_inv))

[[ 1.00e+00  0.00e+00 -4.80e+02  0.00e+00]
 [ 0.00e+00  1.00e+00 -3.60e+02  0.00e+00]
 [ 5.00e-01  3.75e-01 -3.74e+02  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00]]


In [9]:
# Bakul1_100 = [0.999624, -0.0232377, -0.00441006, 0.0138727], [0.0877391, -5.68903, -1.18716]
# Bakul1_130 = [0.999578, -0.0241463, -0.0054248, 0.015219], [0.0443816, -5.26152, -1.21401]
# Bakul1_150 = [0.999429, -0.0298655, -0.00258129, 0.015579], [0.0646409, -5.04149, -0.943123]
# Bakul1_390 = [0.999531, -0.0239092, 0.00874164, 0.0170169], [0.116173, -2.21853, -0.475097]
'''
Input below to quaternion_rotation_matrix() is of a single Image from COLMAP files i.e. Quartenions and Translation matrix  
'''
rotate_matrix, translation_matrix, T = quaternion_rotation_matrix([0.999531, -0.0239092, 0.00874164, 0.0170169], [0.116173, -2.21853, -0.475097])

## 2.950000000000000000e+02,3.700000000000000000e+01,4.600000000000000000e+02,1.900000000000000000e+02
## 2.770000000000000000e+02,6.500000000000000000e+01,4.870000000000000000e+02,2.550000000000000000e+02,6.930000000000000000e+02,6.700000000000000000e+01,8.930000000000000000e+02,2.500000000000000000e+02
## 2.860000000000000000e+02,1.100000000000000000e+02,4.910000000000000000e+02,2.920000000000000000e+02,7.030000000000000000e+02,1.070000000000000000e+02,8.960000000000000000e+02,2.870000000000000000e+02
# Bakul1_390 (1st floor window) = [7.370000000000000000e+02,4.370000000000000000e+02,8.920000000000000000e+02,5.500000000000000000e+02],[3.400000000000000000e+02,4.250000000000000000e+02,4.920000000000000000e+02,5.350000000000000000e+02],[3.150000000000000000e+02,0.000000000000000000e+00,5.290000000000000000e+02,1.350000000000000000e+02],[7.480000000000000000e+02,0.000000000000000000e+00,9.510000000000000000e+02,1.320000000000000000e+02]

'''
Input below is coordinates of windows in an image from our generated CSV files 
'''
sX, sY, eX, eY = 3.150000000000000000e+02,0.000000000000000000e+00,5.290000000000000000e+02,1.350000000000000000e+02

# start3D = np.matmul(R, np.array([[sX], [sY], [0]])) + t
# end3D = np.matmul(R, np.array([[eX], [eY], [0]])) + t

'''
start2D: start 2D(left top) coordinate of window
end2D: end 2D(right bottom) coordinate of window
start3D: start 3D(left top) coordinate of window
end3D: end 3D(right bottom) coordinate of window
'''
start2D = np.array([[sX], [sY], [1]])
end2D = np.array([[eX], [eY], [1]])

start3D = mapto3D(T, K, K_inv.T, start2D)
end3D = mapto3D(T, K, K_inv.T, end2D)

print('start3D:', start3D)
print('end3D:', end3D)

T shape: (4, 4)
R shape: (3, 3)
R_inv shape: (3, 3)
t_inv shape: (3, 1)
K_inv shape: (4, 3)
T shape: (4, 4)
R shape: (3, 3)
R_inv shape: (3, 3)
t_inv shape: (3, 1)
K_inv shape: (4, 3)
start3D: [[   3.19008213]
 [   7.42221003]
 [-156.29164478]
 [   0.        ]]
end3D: [[   6.30033284]
 [  15.04186151]
 [-313.70182086]
 [   0.        ]]


In [7]:
'''
Ground Floor left window:
start3D: [[  -1.34609643]
 [   7.89968922]
 [-161.67688303]
 [   0.        ]]
end3D: [[  -2.90328446]
 [  16.62776661]
 [-337.70409306]
 [   0.        ]]

start3D: [[  -1.00235651]
 [   7.50033853]
 [-160.19669187]
 [   0.        ]]
end3D: [[  -1.96916892]
 [  14.1701656 ]
 [-299.90943928]
 [   0.        ]]

start3D: [[  -0.47362576]
 [  11.05928613]
 [-182.91551128]
 [   0.        ]]
end3D: [[  -0.97637376]
 [  21.44879993]
 [-353.34851168]
 [   0.        ]]
'''


###
'''
start3D: [[293.94412671]
 [ 39.49905013]
 [ -0.49987153]]
end3D: [[454.66211901]
 [196.88502064]
 [ -6.27826438]]
'''

'\nstart3D: [[293.94412671]\n [ 39.49905013]\n [ -0.49987153]]\nend3D: [[454.66211901]\n [196.88502064]\n [ -6.27826438]]\n'